In [1]:
import requests
import pprint
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import pyproj
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap

In [7]:
subway_station_data = pd.read_csv('train_station_info.csv',encoding='cp949')
subway_station_data.head()

,역번호,역사명,노선번호,노선명,영문역사명,한자역사명,환승역구분,환승노선번호,환승노선명,역위도,역경도,운영기관명,역사도로명주소,역사전화번호,데이터기준일자,제공기관코드,제공기관명
0,3122,부평삼거리,S2801,인천지하철 1호선,Bupyeongsamgeori,富平三거리,일반역,-,-,37.478267,126.710494,인천교통공사,인천광역시 부평구 경인로 765,032-516-3122,2020-11-30,B551894,인천교통공사
1,3123,간석오거리,S2801,인천지하철 1호선,Ganseogogeori,間石五거리,일반역,-,-,37.466921,126.707861,인천교통공사,인천광역시 남동구 경인로 674,032-434-3123,2020-11-30,B551894,인천교통공사
2,3124,인천시청,S2801,인천지하철 1호선,Incheon City Hall,仁川市廳,일반역,-,-,37.457615,126.702216,인천교통공사,인천광역시 남동구 예술로 264,032-441-3124,2020-11-30,B551894,인천교통공사
3,3125,예술회관,S2801,인천지하철 1호선,Arts Center,藝術會館,일반역,-,-,37.449334,126.700991,인천교통공사,인천광역시 남동구 예술로 172,032-441-3125,2020-11-30,B551894,인천교통공사
4,3126,인천터미널,S2801,인천지하철 1호선,Incheon Bus Terminal,仁川터미널,일반역,-,-,37.441917,126.699727,인천교통공사,인천광역시 미추홀구 예술로 85,032-441-3126,2020-11-30,B551894,인천교통공사


In [8]:
subway_station_data["역위도"] = subway_station_data["역위도"].astype(float)
subway_station_data["역경도"] = subway_station_data["역경도"].astype(float)

subway_station_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1015 entries, 0 to 1014
Data columns (total 17 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   역번호      1015 non-null   object 
 1   역사명      1015 non-null   object 
 2   노선번호     1015 non-null   object 
 3   노선명      1015 non-null   object 
 4   영문역사명    1015 non-null   object 
 5   한자역사명    1003 non-null   object 
 6   환승역구분    1015 non-null   object 
 7   환승노선번호   236 non-null    object 
 8   환승노선명    275 non-null    object 
 9   역위도      1015 non-null   float64
 10  역경도      1015 non-null   float64
 11  운영기관명    1015 non-null   object 
 12  역사도로명주소  1015 non-null   object 
 13  역사전화번호   1015 non-null   object 
 14  데이터기준일자  1015 non-null   object 
 15  제공기관코드   1015 non-null   object 
 16  제공기관명    1015 non-null   object 
dtypes: float64(2), object(15)
memory usage: 134.9+ KB


In [10]:
center = [35.8714354, 128.601445]

In [18]:
base_map = folium.Map(location=center,zoom_start=12)
base_map

In [19]:
for index_num in range(len(subway_station_data)):
    folium.Circle(
        location=[subway_station_data.loc[index_num, "역위도"], subway_station_data.loc[index_num, "역경도"]],
        radius=50,
        color="blue",
        popup=subway_station_data.loc[index_num, "역사명"]
    ).add_to(base_map)

base_map

In [20]:
bus_line_data = pd.read_csv("20190124_seoul_bus.csv")
bus_line_data.head(10)

,노선ID,노선명,순번,구간ID,정류소ID,ARS-ID,정류소명,X좌표,Y좌표
0,100100124,0017,1,0,102000271,3689,청암자이아파트,126.946517,37.534363
1,100100124,0017,2,102700549,102000204,3298,청암동강변삼성아파트,126.949304,37.533961
2,100100124,0017,3,102700550,102000227,3321,청심경로당,126.950449,37.533744
3,100100124,0017,4,102700551,102000210,3304,원효2동주민센터,126.950904,37.534278
4,100100124,0017,5,102700552,102000212,3306,산천동,126.953984,37.535420
5,100100124,0017,6,102700553,102000214,3308,신창동세방아파트,126.955032,37.535749
6,100100124,0017,7,102700554,102000216,3310,남이장군사당,126.957957,37.536707
7,100100124,0017,8,102700555,102000218,3312,새마을금고,126.959503,37.537126
8,100100124,0017,9,102700779,102000153,3247,용문시장,126.960735,37.536630
9,100100124,0017,10,102700557,102000156,3250,용산전자상가입구,126.962043,37.534483


In [21]:
sample_bus_data = bus_line_data[bus_line_data['노선명'] == '0017']
# index 초기화
sample_bus_data =sample_bus_data.reset_index()

In [22]:
sample_bus_data

,index,노선ID,노선명,순번,구간ID,정류소ID,ARS-ID,정류소명,X좌표,Y좌표
0,0,100100124,0017,1,0,102000271,3689,청암자이아파트,126.946517,37.534363
1,1,100100124,0017,2,102700549,102000204,3298,청암동강변삼성아파트,126.949304,37.533961
2,2,100100124,0017,3,102700550,102000227,3321,청심경로당,126.950449,37.533744
3,3,100100124,0017,4,102700551,102000210,3304,원효2동주민센터,126.950904,37.534278
4,4,100100124,0017,5,102700552,102000212,3306,산천동,126.953984,37.535420
5,5,100100124,0017,6,102700553,102000214,3308,신창동세방아파트,126.955032,37.535749
6,6,100100124,0017,7,102700554,102000216,3310,남이장군사당,126.957957,37.536707
7,7,100100124,0017,8,102700555,102000218,3312,새마을금고,126.959503,37.537126
8,8,100100124,0017,9,102700779,102000153,3247,용문시장,126.960735,37.536630
9,9,100100124,0017,10,102700557,102000156,3250,용산전자상가입구,126.962043,37.534483


In [24]:
sample_point_list = [[sample_bus_data.loc[index_num, 'Y좌표'], sample_bus_data.loc[index_num, 'X좌표']] for index_num in range(len(sample_bus_data))]
sample_point_list

[[37.5343626071, 126.9465174884],
 [37.5339606747, 126.9493037945],
 [37.5337439118, 126.9504485667],
 [37.5342784061, 126.9509043881],
 [37.5354201502, 126.9539838422],
 [37.5357487714, 126.9550322215],
 [37.5367069612, 126.9579568939],
 [37.5371259582, 126.9595027769],
 [37.5366304221, 126.9607348708],
 [37.5344828174, 126.9620433894],
 [37.5326572802, 126.963900286],
 [37.529349, 126.966155],
 [37.527193494, 126.9645186276],
 [37.5252500866, 126.964005685],
 [37.5236972645, 126.959676039],
 [37.5261482683, 126.9562789235],
 [37.5277168019, 126.9550134],
 [37.5301735631, 126.9545557512],
 [37.5301685887, 126.9530395553],
 [37.5324430889, 126.9495515487],
 [37.5339606747, 126.9493037945],
 [37.5330803284, 126.9505883522],
 [37.5322407924, 126.9535516914],
 [37.5297428608, 126.9544433676],
 [37.5279691953, 126.9547629377],
 [37.5260689531, 126.9562314358],
 [37.5235147022, 126.9598438074],
 [37.5241009704, 126.9631731011],
 [37.5272804289, 126.9661443758],
 [37.5299861801, 126.96763021

In [26]:
seoul_map = folium.Map(location=[37.541, 126.986], zoom_start=13)
seoul_map

In [27]:
folium.PolyLine(
    locations=sample_point_list,
    color="blue",
    weight=2.5,   # 선의 굵기
    opacity=0.5  # 0~1 선명도를 뜻함
).add_to(seoul_map)

seoul_map

In [29]:
seoul_sig = json.load(open("korea_seoul_sig.json", encoding='utf-8'))

In [30]:
seoul_map2 = folium.Map(location=[37.541, 126.986], zoom_start=10)

In [31]:
folium.GeoJson(seoul_sig,name='seoul_gu').add_to(seoul_map2)
seoul_map2